In [356]:
import pandas as pd

In [357]:
matches = pd.read_csv("VNL_mens_stats.csv", index_col=0)

In [358]:
matches.head()

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,set5,attacks,blocks,serves,digs,receives,sets
1,16128,Bulgaria,China,L,2,3,20,25,21,25.0,NaN,52,10,4,39,83,106
2,16128,China,Bulgaria,W,3,2,25,21,25,20.0,5.0,56,12,8,32,84,98
3,16129,Japan,Iran,W,3,0,25,25,25,NaN,NaN,47,4,4,48,45,83
4,16129,Iran,Japan,L,0,3,16,22,19,NaN,NaN,29,6,3,26,61,78
5,16130,Italy,Argentina,L,0,3,22,23,18,NaN,NaN,37,2,2,51,56,87


In [359]:
matches.head()

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,set5,attacks,blocks,serves,digs,receives,sets
1,16128,Bulgaria,China,L,2,3,20,25,21,25.0,NaN,52,10,4,39,83,106
2,16128,China,Bulgaria,W,3,2,25,21,25,20.0,5.0,56,12,8,32,84,98
3,16129,Japan,Iran,W,3,0,25,25,25,NaN,NaN,47,4,4,48,45,83
4,16129,Iran,Japan,L,0,3,16,22,19,NaN,NaN,29,6,3,26,61,78
5,16130,Italy,Argentina,L,0,3,22,23,18,NaN,NaN,37,2,2,51,56,87


In [360]:
matches.shape

(648, 17)

In [361]:
matches["team"].value_counts()

team
Poland         45
Japan          43
Slovenia       43
Italy          43
France         41
Brazil         41
USA            39
Argentina      38
Canada         37
Cuba           37
Netherlands    36
Bulgaria       36
Serbia         36
Iran           36
Germany        36
China          25
Türkiye        24
Ukraine        12
Name: count, dtype: int64

In [362]:
matches.dtypes

matchid        int64
team          object
opponent      object
result        object
sets won       int64
sets lost      int64
set1           int64
set2           int64
set3           int64
set4         float64
set5         float64
attacks        int64
blocks         int64
serves         int64
digs           int64
receives       int64
sets           int64
dtype: object

In [363]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["target"] = (matches["result"] == "W").astype("int")

In [364]:
matches

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,set5,attacks,blocks,serves,digs,receives,sets,opp_code,target
1,16128,Bulgaria,China,L,2,3,20,25,21,25.0,NaN,52,10,4,39,83,106,4,0
2,16128,China,Bulgaria,W,3,2,25,21,25,20.0,5.0,56,12,8,32,84,98,2,1
3,16129,Japan,Iran,W,3,0,25,25,25,NaN,NaN,47,4,4,48,45,83,8,1
4,16129,Iran,Japan,L,0,3,16,22,19,NaN,NaN,29,6,3,26,61,78,10,0
5,16130,Italy,Argentina,L,0,3,22,23,18,NaN,NaN,37,2,2,51,56,87,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,21550,Slovenia,Italy,L,1,3,22,25,21,18.0,NaN,52,9,2,64,79,106,9,0
645,21551,Brazil,Slovenia,W,3,1,23,25,25,25.0,NaN,48,11,5,60,68,110,14,1
646,21551,Slovenia,Brazil,L,1,3,25,20,23,19.0,NaN,49,7,2,68,80,122,1,0
647,21552,Poland,Italy,W,3,0,25,25,25,NaN,NaN,37,10,3,55,43,80,9,1


In [365]:
from sklearn.ensemble import RandomForestClassifier

In [366]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [367]:
# edit this line to change what subset of the data to train on
train = matches[matches["matchid"] < 21000]

In [368]:
# edit this line to change what subset of the data to test
test = matches[matches["matchid"] > 21000]

In [369]:
predictors = ["opp_code"]

In [370]:
rf.fit(train[predictors], train["target"])

,n_estimators,50
,criterion,'gini'
,max_depth,None
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [371]:
preds = rf.predict(test[predictors])

In [372]:
from sklearn.metrics import accuracy_score

In [373]:
acc = accuracy_score(test["target"], preds)

In [374]:
acc

0.646551724137931

In [375]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [376]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,78,38
1,44,72


In [377]:
from sklearn.metrics import precision_score

In [378]:
precision_score(test["target"], preds)

0.6545454545454545

In [379]:
# one dataframe for every team in the dataset
# rolling averages; "how did team A do in the previous x matches?" to predict how they will do on their next match
grouped_matches = matches.groupby("team")

In [380]:
# group = grouped_matches.get_group("Poland")

In [381]:
# group

In [382]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("matchid")
    rolling_stats = group[cols].rolling(3, closed='left').mean() # closed='left' means take the current match out when creating rolling averages
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols) # removes rows that have missing values
    return group

In [383]:
cols = ["sets won", "sets lost", "attacks", "blocks", "serves", "digs", "receives", "sets"]
new_cols = [f"{c}_rolling" for c in cols]

In [384]:
new_cols

['sets won_rolling',
 'sets lost_rolling',
 'attacks_rolling',
 'blocks_rolling',
 'serves_rolling',
 'digs_rolling',
 'receives_rolling',
 'sets_rolling']

In [385]:
rolling_averages(group, cols, new_cols)

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,...,opp_code,target,sets won_rolling,sets lost_rolling,attacks_rolling,blocks_rolling,serves_rolling,digs_rolling,receives_rolling,sets_rolling
56,16155,Poland,Serbia,L,0,3,21,19,14,NaN,...,13,0,3.000000,1.666667,58.666667,10.333333,7.666667,48.666667,76.000000,107.000000
86,16170,Poland,Germany,W,3,2,25,22,25,17.0,...,7,1,2.000000,2.333333,53.000000,7.333333,6.000000,46.000000,72.000000,102.666667
98,16176,Poland,Netherlands,W,3,2,22,25,18,25.0,...,11,1,2.000000,2.333333,50.666667,8.666667,6.000000,47.000000,72.333333,105.666667
111,16183,Poland,USA,L,0,3,22,18,19,NaN,...,16,0,2.000000,2.333333,50.666667,9.000000,4.666667,42.333333,73.000000,98.333333
122,16188,Poland,Italy,W,3,1,25,28,18,25.0,...,9,1,2.000000,2.333333,48.333333,9.666667,4.333333,46.666667,71.000000,96.666667
139,16197,Poland,Slovenia,W,3,2,29,21,25,25.0,...,14,1,2.000000,2.000000,47.333333,7.333333,4.666667,48.666667,68.333333,92.666667
157,16206,Poland,Brazil,W,3,1,25,22,25,25.0,...,1,1,2.000000,2.000000,50.333333,7.666667,5.000000,60.666667,74.666667,110.333333
171,16213,Poland,Canada,W,3,0,25,25,27,NaN,...,3,1,3.000000,1.333333,58.333333,8.000000,5.333333,61.333333,78.333333,116.333333
186,16220,Poland,Japan,W,3,0,25,25,25,NaN,...,10,1,3.000000,1.000000,53.333333,9.000000,4.000000,55.333333,73.333333,106.000000
199,16227,Poland,Brazil,W,3,0,26,25,25,NaN,...,1,1,3.000000,0.333333,46.000000,8.333333,4.666667,43.333333,57.000000,83.333333


In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [387]:
matches_rolling

matchid       team     opponent result  sets won  sets lost  \
team                                                                         
Argentina 59     16157  Argentina  Netherlands      W         3          2   
          71     16163  Argentina     Slovenia      L         1          3   
          87     16171  Argentina       France      W         3          1   
          101    16178  Argentina     Bulgaria      W         3          0   
          110    16182  Argentina        Japan      L         2          3   
...                ...        ...          ...    ...       ...        ...   
Ukraine   552    21504    Ukraine     Bulgaria      W         3          1   
          571    21514    Ukraine  Netherlands      W         3          2   
          587    21522    Ukraine        Italy      L         2          3   
          599    21528    Ukraine       Serbia      L         0          3   
          621    21539    Ukraine       Canada      L         1          3   

               set1  set2  set3  set4  ...  opp_code  target  \
team                                   ...                     
Argentina 59     34    25    25  20.0  ...        11       1   
          71     25    21    21  21.0  ...        14       0   
          87     22    26    25  25.0  ...         6       1   
          101    25    25    25   NaN  ...         2       1   
          110    18    22    33  25.0  ...        10       0   
...             ...   ...   ...   ...  ...       ...     ...   
Ukraine   552    20    25    26  25.0  ...         2       1   
          571    25    20    20  26.0  ...        11       1   
          587    15    20    25  25.0  ...         9       0   
          599    22    19    17   NaN  ...        13       0   
          621    21    27    29  21.0  ...         3       0   

               sets won_rolling  sets lost_rolling  attacks_rolling  \
team                                                                  
Argentina 59           2.666667           1.333333        51.666667   
          71           2.666667           2.000000        59.333333   
          87           2.333333           2.000000        57.333333   
          101          2.333333           2.000000        56.000000   
          110          2.333333           1.333333        49.000000   
...                         ...                ...              ...   
Ukraine   552          2.000000           1.666667        45.333333   
          571          3.000000           1.000000        52.666667   
          587          3.000000           1.666667        58.666667   
          599          2.666667           2.000000        58.000000   
          621          1.666667           2.666667        51.666667   

               blocks_rolling  serves_rolling  digs_rolling  receives_rolling  \
team                                                                            
Argentina 59         9.333333        3.666667     69.333333         71.666667   
          71        11.666667        5.000000     86.333333         84.000000   
          87         9.333333        4.333333     75.666667         79.000000   
          101        8.333333        4.666667     64.666667         78.000000   
          110        5.666667        4.333333     55.333333         64.333333   
...                       ...             ...           ...               ...   
Ukraine   552        9.666667        4.333333     54.666667         62.333333   
          571       11.000000        5.333333     63.000000         66.666667   
          587        9.666667        6.000000     71.000000         80.000000   
          599        8.666667        5.333333     70.000000         84.333333   
          621        7.333333        4.000000     61.000000         82.000000   

               sets_rolling  
team                         
Argentina 59     108.666667  
          71     128.666667  
          87     121.333333  
          101    116.000000  
         

In [388]:
matches_rolling = matches_rolling.droplevel('team')

In [389]:
matches_rolling

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,...,opp_code,target,sets won_rolling,sets lost_rolling,attacks_rolling,blocks_rolling,serves_rolling,digs_rolling,receives_rolling,sets_rolling
59,16157,Argentina,Netherlands,W,3,2,34,25,25,20.0,...,11,1,2.666667,1.333333,51.666667,9.333333,3.666667,69.333333,71.666667,108.666667
71,16163,Argentina,Slovenia,L,1,3,25,21,21,21.0,...,14,0,2.666667,2.000000,59.333333,11.666667,5.000000,86.333333,84.000000,128.666667
87,16171,Argentina,France,W,3,1,22,26,25,25.0,...,6,1,2.333333,2.000000,57.333333,9.333333,4.333333,75.666667,79.000000,121.333333
101,16178,Argentina,Bulgaria,W,3,0,25,25,25,NaN,...,2,1,2.333333,2.000000,56.000000,8.333333,4.666667,64.666667,78.000000,116.000000
110,16182,Argentina,Japan,L,2,3,18,22,33,25.0,...,10,0,2.333333,1.333333,49.000000,5.666667,4.333333,55.333333,64.333333,98.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,21504,Ukraine,Bulgaria,W,3,1,20,25,26,25.0,...,2,1,2.000000,1.666667,45.333333,9.666667,4.333333,54.666667,62.333333,89.666667
571,21514,Ukraine,Netherlands,W,3,2,25,20,20,26.0,...,11,1,3.000000,1.000000,52.666667,11.000000,5.333333,63.000000,66.666667,104.333333
587,21522,Ukraine,Italy,L,2,3,15,20,25,25.0,...,9,0,3.000000,1.666667,58.666667,9.666667,6.000000,71.000000,80.000000,118.000000
599,21528,Ukraine,Serbia,L,0,3,22,19,17,NaN,...,13,0,2.666667,2.000000,58.000000,8.666667,5.333333,70.000000,84.333333,122.000000


In [390]:
matches_rolling.index = range(matches_rolling.shape[0])

In [391]:
matches_rolling

,matchid,team,opponent,result,sets won,sets lost,set1,set2,set3,set4,...,opp_code,target,sets won_rolling,sets lost_rolling,attacks_rolling,blocks_rolling,serves_rolling,digs_rolling,receives_rolling,sets_rolling
0,16157,Argentina,Netherlands,W,3,2,34,25,25,20.0,...,11,1,2.666667,1.333333,51.666667,9.333333,3.666667,69.333333,71.666667,108.666667
1,16163,Argentina,Slovenia,L,1,3,25,21,21,21.0,...,14,0,2.666667,2.000000,59.333333,11.666667,5.000000,86.333333,84.000000,128.666667
2,16171,Argentina,France,W,3,1,22,26,25,25.0,...,6,1,2.333333,2.000000,57.333333,9.333333,4.333333,75.666667,79.000000,121.333333
3,16178,Argentina,Bulgaria,W,3,0,25,25,25,NaN,...,2,1,2.333333,2.000000,56.000000,8.333333,4.666667,64.666667,78.000000,116.000000
4,16182,Argentina,Japan,L,2,3,18,22,33,25.0,...,10,0,2.333333,1.333333,49.000000,5.666667,4.333333,55.333333,64.333333,98.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,21504,Ukraine,Bulgaria,W,3,1,20,25,26,25.0,...,2,1,2.000000,1.666667,45.333333,9.666667,4.333333,54.666667,62.333333,89.666667
590,21514,Ukraine,Netherlands,W,3,2,25,20,20,26.0,...,11,1,3.000000,1.000000,52.666667,11.000000,5.333333,63.000000,66.666667,104.333333
591,21522,Ukraine,Italy,L,2,3,15,20,25,25.0,...,9,0,3.000000,1.666667,58.666667,9.666667,6.000000,71.000000,80.000000,118.000000
592,21528,Ukraine,Serbia,L,0,3,22,19,17,NaN,...,13,0,2.666667,2.000000,58.000000,8.666667,5.333333,70.000000,84.333333,122.000000


In [413]:
def make_predictions(data, predictors):
    train = data[data["matchid"] < 21000]
    test = data[data["matchid"] > 21000]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [414]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [415]:
precision

0.5419847328244275

In [416]:
combined

,actual,predicted
23,1,0
24,1,1
25,0,1
26,0,0
27,1,1
...,...,...
589,1,1
590,1,1
591,0,1
592,0,1


In [417]:
combined = combined.merge(matches_rolling[["matchid", "team", "opponent", "result"]], left_index=True, right_index=True)

In [418]:
combined

,actual,predicted,matchid,team,opponent,result
23,1,0,21442,Argentina,France,W
24,1,1,21450,Argentina,Canada,W
25,0,1,21454,Argentina,Bulgaria,L
26,0,0,21468,Argentina,Italy,L
27,1,1,21476,Argentina,Netherlands,W
...,...,...,...,...,...,...
589,1,1,21504,Ukraine,Bulgaria,W
590,1,1,21514,Ukraine,Netherlands,W
591,0,1,21522,Ukraine,Italy,L
592,0,1,21528,Ukraine,Serbia,L


In [419]:
merged = combined.merge(combined, left_on=["matchid", "team"], right_on=["matchid", "opponent"])

In [420]:
merged

,actual_x,predicted_x,matchid,team_x,opponent_x,result_x,actual_y,predicted_y,team_y,opponent_y,result_y
0,1,0,21442,Argentina,France,W,0,0,France,Argentina,L
1,1,1,21450,Argentina,Canada,W,0,0,Canada,Argentina,L
2,0,1,21454,Argentina,Bulgaria,L,1,0,Bulgaria,Argentina,W
3,0,0,21468,Argentina,Italy,L,1,1,Italy,Argentina,W
4,1,1,21476,Argentina,Netherlands,W,0,0,Netherlands,Argentina,L
...,...,...,...,...,...,...,...,...,...,...,...
221,1,1,21504,Ukraine,Bulgaria,W,0,0,Bulgaria,Ukraine,L
222,1,1,21514,Ukraine,Netherlands,W,0,1,Netherlands,Ukraine,L
223,0,1,21522,Ukraine,Italy,L,1,1,Italy,Ukraine,W
224,0,1,21528,Ukraine,Serbia,L,1,0,Serbia,Ukraine,W


In [421]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

actual_x
1    37
0    26
Name: count, dtype: int64

In [423]:
37/(37+26)

0.5873015873015873